# Generate Data

In [ ]:
import tensorflow.keras as keras

In [ ]:
# Módulos necesarios. Asegurarse de poder importarlos.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import sys
from six.moves import cPickle as pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import csv
from pylab import rcParams
from scipy import stats
from sklearn import metrics
import seaborn as sns
from sklearn import preprocessing
import os as os
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
tf.__version__

/usr/bin/python
3.6.8 (default, Aug 20 2019, 17:12:48) 
[GCC 8.3.0]
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)


'1.14.0'

In [ ]:
keras.__version__

'2.2.4-tf'

# Dataset generation

I make use of the files that contain the quaternions differnatiated per subject and activity.

I define a number of functions to read the data and finally generated the dataset 'on the fly'.

In [ ]:
# Root path
PATH = '../data_reset_def_raworientation_ALL_SENSORS'

#Checkpoints path
CKPATH = PATH + '/checkpoints'

quaturls = !ls -1 "{PATH}"

In [ ]:
n_act = 11

In [ ]:
global activities
activities = [9,10,11,12,13,19,20,21,24,25,31]
activities = activities[:n_act]

In [ ]:
quat_corr = []
for filename in quaturls:
    for activity in activities:
        if(int(filename[-6:-4])==activity):
            quat_corr.append(filename)

In [ ]:
global subjects
subjects = [1,2,3,5,8,9,10,11,13,14,16,17]

In [ ]:
quaturls = []

i=0
for filename in quat_corr:
    for subject in subjects:
        if(int(quat_corr[i][8:10])==subject):
            quaturls.append(filename)
    i+=1

### Differentiating per subject

In [ ]:
print('Total files: ' + str(len(quaturls)))

Total files: 132


In [ ]:
data_fullpath = [os.path.join(PATH,s) for s in quaturls]
print('DATA: ')
print(data_fullpath[:10])

DATA: 
['../data_reset_def_raworientation_ALL_SENSORS/subject_01_RESET_RAW_act_09.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_01_RESET_RAW_act_10.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_01_RESET_RAW_act_11.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_01_RESET_RAW_act_12.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_01_RESET_RAW_act_13.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_01_RESET_RAW_act_19.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_01_RESET_RAW_act_20.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_01_RESET_RAW_act_21.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_01_RESET_RAW_act_24.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_01_RESET_RAW_act_25.csv']


### Class balancing

For class balancing we make each class have the same number of samples, so we limit it to the minimum-class number (there will be data augmentation afterwards):

In [ ]:
n = 1e1000
for filename in data_fullpath:
    df = pd.read_csv(filename,sep=',')

    ind = len(df.index)
    if(ind!=0 and ind<n):
        n = ind
        print(filename, end='\r')

ind_tr = int(n)
if(PATH == '../data_augment'):
    ind_real = 24*int(n/9)
else:
    ind_real = int(n/9)


In [ ]:
print('Tenemos: '+ str(ind_real) + ' muestras de cada actividad de cada sujeto, que se corresponde con: '+ str(ind_real/50) + ' segundos.')

Tenemos: 401 muestras de cada actividad de cada sujeto, que se corresponde con: 8.02 segundos.


In [ ]:
print(ind_tr)

3609


### Differentiating per subject

In [ ]:
s1, s2, s3, s5, s8, s9, s10, s11, s13, s14, s16, s17 = [], [], [], [], [], [], [], [], [], [], [], []

i=0
for filename in data_fullpath:
    if(int(data_fullpath[i][53:55])==1):
        s1.append(filename)
    elif(int(data_fullpath[i][53:55])==2):
        s2.append(filename)
    elif(int(data_fullpath[i][53:55])==3):
        s3.append(filename)
    elif(int(data_fullpath[i][53:55])==5):
        s5.append(filename)
    elif(int(data_fullpath[i][53:55])==8):
        s8.append(filename)
    elif(int(data_fullpath[i][53:55])==9):
        s9.append(filename)
    elif(int(data_fullpath[i][53:55])==10):
        s10.append(filename)
    elif(int(data_fullpath[i][53:55])==11):
        s11.append(filename)
    elif(int(data_fullpath[i][53:55])==13):
        s13.append(filename)
    elif(int(data_fullpath[i][53:55])==14):
        s14.append(filename)
    elif(int(data_fullpath[i][53:55])==16):
        s16.append(filename)
    elif(int(data_fullpath[i][53:55])==17):
        s17.append(filename)

    i+=1

In [ ]:
print(s1[:2], s2[:2], s3[:2], s5[:2], s8[:2], s9[:2], s10[:2], s11[:2], s13[:2], s14[:2], s16[:2], s17[:2])

['../data_reset_def_raworientation_ALL_SENSORS/subject_01_RESET_RAW_act_09.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_01_RESET_RAW_act_10.csv'] ['../data_reset_def_raworientation_ALL_SENSORS/subject_02_RESET_RAW_act_09.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_02_RESET_RAW_act_10.csv'] ['../data_reset_def_raworientation_ALL_SENSORS/subject_03_RESET_RAW_act_09.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_03_RESET_RAW_act_10.csv'] ['../data_reset_def_raworientation_ALL_SENSORS/subject_05_RESET_RAW_act_09.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_05_RESET_RAW_act_10.csv'] ['../data_reset_def_raworientation_ALL_SENSORS/subject_08_RESET_RAW_act_09.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_08_RESET_RAW_act_10.csv'] ['../data_reset_def_raworientation_ALL_SENSORS/subject_09_RESET_RAW_act_09.csv', '../data_reset_def_raworientation_ALL_SENSORS/subject_09_RESET_RAW_act_10.csv'] ['../data_reset_def_raworientation

### Working with lists and numpy (heavier but helps to keep dimensions)

In [ ]:
def sliding_window(df, n_time_steps, step, segments, labels, label, dim, train, n_channels):
    quat0 = df.iloc[:, 1:5][df['QUAT']=='quat0'].reset_index() # si no incluimos el reset_index(), al concatenarlos después
    quat1 = df.iloc[:, 1:5][df['QUAT']=='quat1'].reset_index() # aparecerá un dataframe de igual nº de filas que df, pero con
    quat2 = df.iloc[:, 1:5][df['QUAT']=='quat2'].reset_index() # NaN en las posiciones que no tienen número de cada dataframe quat,
    quat3 = df.iloc[:, 1:5][df['QUAT']=='quat3'].reset_index() # es decir, mantiene los índices de df.
    quat4 = df.iloc[:, 1:5][df['QUAT']=='quat4'].reset_index()
    quat5 = df.iloc[:, 1:5][df['QUAT']=='quat5'].reset_index()
    quat6 = df.iloc[:, 1:5][df['QUAT']=='quat6'].reset_index()
    quat7 = df.iloc[:, 1:5][df['QUAT']=='quat7'].reset_index()
    quat8 = df.iloc[:, 1:5][df['QUAT']=='quat8'].reset_index()

    quat = pd.concat([quat0.iloc[:, 1], quat0.iloc[:, 2], quat0.iloc[:, 3], quat0.iloc[:, 4],
                      quat1.iloc[:, 1], quat1.iloc[:, 2], quat1.iloc[:, 3], quat1.iloc[:, 4],
                      quat2.iloc[:, 1], quat2.iloc[:, 2], quat2.iloc[:, 3], quat2.iloc[:, 4],
                      quat3.iloc[:, 1], quat3.iloc[:, 2], quat3.iloc[:, 3], quat3.iloc[:, 4],
                      quat4.iloc[:, 1], quat4.iloc[:, 2], quat4.iloc[:, 3], quat4.iloc[:, 4],
                      quat5.iloc[:, 1], quat5.iloc[:, 2], quat5.iloc[:, 3], quat5.iloc[:, 4],
                      quat6.iloc[:, 1], quat6.iloc[:, 2], quat6.iloc[:, 3], quat6.iloc[:, 4],
                      quat7.iloc[:, 1], quat7.iloc[:, 2], quat7.iloc[:, 3], quat7.iloc[:, 4],
                      quat8.iloc[:, 1], quat8.iloc[:, 2], quat8.iloc[:, 3], quat8.iloc[:, 4]],
                      axis = 1, keys = ['w0', 'x0', 'y0', 'z0', 'w1', 'x1', 'y1', 'z1', 'w2', 'x2', 'y2', 'z2',
                                        'w3', 'x3', 'y3', 'z3', 'w4', 'x4', 'y4', 'z4', 'w5', 'x5', 'y5', 'z5',
                                        'w6', 'x6', 'y6', 'z6', 'w7', 'x7', 'y7', 'z7', 'w8', 'x8', 'y8', 'z8'])
    del quat0, quat1, quat2, quat3, quat4, quat5, quat6, quat7, quat8

    if(train==False):
        step = n_time_steps//2

    if(n_channels == 1):
        for i in range(0, quat.shape[0] - n_time_steps, step): # Overlap
            # Con listas y numpy
            segments.append([])
            segments[dim].append(quat.iloc[i: i + n_time_steps, :].values) # Si distinguimos entre sensores
            labels.append(int(label-1))
            dim+=1
    else:
        n_columns = int(36/n_channels)
        for i in range(0, quat.shape[0] - n_time_steps, step):
            segments.append([])
            col = 0
            for j in range(n_channels):
                segments[dim].append([])
                segments[dim][j].append(quat.iloc[i:i+n_time_steps,col:col+n_columns].values)
                col+=n_columns
            labels.append(label-1)
            dim+=1


    del quat

    return segments, labels, dim

In [ ]:
def load_quat(path, lim, train, n_channels):

    step = n_time_steps//4  # n_time_steps/50 segundos de actividad y pasos de step/50 segundos (de overlap)

    segments = []
    labels = []

    i=1
    dim = 0
    for filename in path:
        print("Reading %s (%d/%d)                                                   "%(filename, i, len(path)), end='\r')
        df = pd.read_csv(filename,sep=',',names=["QUAT","w","x","y","z","timestamp"])
        label = int(filename[-6:-4])
        for k in range(len(activities)):
            if(label==activities[k]):
                label=k+1

        segments, labels, dim = sliding_window(df.iloc[:lim], n_time_steps, step, segments, labels, label, dim, train, n_channels)

        i+=1

    del df

    return segments, labels


def load_train_quat(filename, lim, n_channels):
    return load_quat(filename, lim, True, n_channels)

def load_test_quat(filename, lim, n_channels):
    return load_quat(filename, lim, False, n_channels)


In [ ]:
def get_dataset(data_path, lim, batch_size, n_time_steps, train, valid, ds, n_channels):
    if(train):
        segments, labels = load_train_quat(data_path, lim, n_channels)
    else:
        segments, labels = load_test_quat(data_path, lim, n_channels)

    print('Generating the dataset                                                   ')

    array = np.asarray(segments, dtype = 'float32')
    segments = np.reshape(array, (array.shape[0], n_channels, n_time_steps, int(36/n_channels)))
    array = np.asarray(labels, dtype = 'int8')
    labels = np.reshape(array, (array.shape[0], 1))

    del array

    # Map coninous dataset to categorical (One-Hot)
    labels = keras.utils.to_categorical(labels, len(activities))

    if(train):
        print('-'*20 + 'TRAIN' + '-'*20)
    elif(valid):
        print('-'*18 + 'VALIDATION' + '-'*17)
    else:
        print('-'*20 + 'TEST' + '-'*21)

    if(ds):
        dataset = tf.data.Dataset.from_tensor_slices((segments, labels))

        # It's necessary to repeat our data for all epochs
        dataset = dataset.batch(batch_size)

        dataset = dataset.shuffle(segments.shape[0])

        print('Dataset generated                                                        ')

        return dataset, segments, labels
    else:
        np.random.seed(235)
        # Shuffle in the first dimension
        permutation = np.arange(0,segments.shape[0]-1)
        np.random.shuffle(permutation)
        segments = segments[permutation]
        labels = labels[permutation]
        print('Dataset generated                                                        ')

        return segments, labels

In [ ]:
global n_time_steps
n_time_steps = 128

def generator(s):
    dataset = False # Set to True if you want a dataset or to False if you want np.arrays
    batch_size = 32 # REAL batch_size
    n_channels = 1 # It can be 1,4 or 9

    subj = s[0][53:55]

    seg, lab = get_dataset(s, ind_tr, batch_size, n_time_steps, True, False, dataset, n_channels)

    print(seg.shape, lab.shape)

    np.save('./def_raworientation_ALL_SENSORS/subject_video_'+subj+'_seg.npy', seg)
    np.save('./def_raworientation_ALL_SENSORS/subject_video_'+subj+'_lab.npy', lab)

    print('Files from subject ' + subj + ' saved')

In [ ]:
generator(s1)

Generating the dataset                                                   _act_31.csv (11/11)                                                   
--------------------TRAIN--------------------
Dataset generated                                                        
(98, 1, 128, 36) (98, 11)
Files from subject 01 saved


In [ ]:
generator(s2)

Generating the dataset                                                   _act_31.csv (11/11)                                                   
--------------------TRAIN--------------------
Dataset generated                                                        
(98, 1, 128, 36) (98, 11)
Files from subject 02 saved


In [ ]:
generator(s3)

Generating the dataset                                                   _act_31.csv (11/11)                                                   
--------------------TRAIN--------------------
Dataset generated                                                        
(98, 1, 128, 36) (98, 11)
Files from subject 03 saved


In [ ]:
generator(s5)

Generating the dataset                                                   _act_31.csv (11/11)                                                   
--------------------TRAIN--------------------
Dataset generated                                                        
(98, 1, 128, 36) (98, 11)
Files from subject 05 saved


In [ ]:
generator(s8)

Generating the dataset                                                   _act_31.csv (11/11)                                                   
--------------------TRAIN--------------------
Dataset generated                                                        
(98, 1, 128, 36) (98, 11)
Files from subject 08 saved


In [ ]:
generator(s9)

Generating the dataset                                                   _act_31.csv (11/11)                                                   
--------------------TRAIN--------------------
Dataset generated                                                        
(98, 1, 128, 36) (98, 11)
Files from subject 09 saved


In [ ]:
generator(s10)

Generating the dataset                                                   _act_31.csv (11/11)                                                   
--------------------TRAIN--------------------
Dataset generated                                                        
(98, 1, 128, 36) (98, 11)
Files from subject 10 saved


In [ ]:
generator(s11)

Generating the dataset                                                   _act_31.csv (11/11)                                                   
--------------------TRAIN--------------------
Dataset generated                                                        
(98, 1, 128, 36) (98, 11)
Files from subject 11 saved


In [ ]:
generator(s13)

Generating the dataset                                                   _act_31.csv (11/11)                                                   
--------------------TRAIN--------------------
Dataset generated                                                        
(98, 1, 128, 36) (98, 11)
Files from subject 13 saved


In [ ]:
generator(s14)

Generating the dataset                                                   _act_31.csv (11/11)                                                   
--------------------TRAIN--------------------
Dataset generated                                                        
(98, 1, 128, 36) (98, 11)
Files from subject 14 saved


In [ ]:
generator(s16)

Generating the dataset                                                   _act_31.csv (11/11)                                                   
--------------------TRAIN--------------------
Dataset generated                                                        
(98, 1, 128, 36) (98, 11)
Files from subject 16 saved


In [ ]:
generator(s17)

Generating the dataset                                                   _act_31.csv (11/11)                                                   
--------------------TRAIN--------------------
Dataset generated                                                        
(98, 1, 128, 36) (98, 11)
Files from subject 17 saved


In [ ]:
print('Operación terminada.')

Operación terminada.
